# HW2: Language Modeling - Linear Interpolation, NNLM, LSTM

## Setup

In [0]:
!pip install -q torch torchtext opt_einsum
!pip install -qU git+https://github.com/harvardnlp/namedtensor

In [0]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.distributions import Dirichlet
from torch.nn.utils import clip_grad_norm_

import re
import math
from collections import Counter
from tqdm import tqdm
from itertools import islice
import numpy as np

import torchtext
from torchtext.vocab import Vectors
from torchtext.data.iterator import BPTTIterator
from torchtext.data import Batch, Dataset

from namedtensor import ntorch
from namedtensor.text import NamedField

In [0]:
# Our input $x$
TEXT = NamedField(names=("seqlen",))

In [0]:
# Download data
!curl -qO https://raw.githubusercontent.com/harvard-ml-courses/cs287-s18/master/HW2/input.txt
!curl -qO https://raw.githubusercontent.com/harvard-ml-courses/cs287-s18/master/HW2/train.5k.txt
!curl -qO https://raw.githubusercontent.com/harvard-ml-courses/cs287-s18/master/HW2/train.txt
!curl -qO https://raw.githubusercontent.com/harvard-ml-courses/cs287-s18/master/HW2/valid.txt

In [0]:
# Data distributed with the assignment
train, val, test = torchtext.datasets.LanguageModelingDataset.splits(
    path=".", 
    train="train.txt", validation="valid.txt", test="valid.txt", text_field=TEXT)

In [0]:
# Build vocab
TEXT.build_vocab(train)

In [0]:
class NamedBpttIterator(BPTTIterator):
    def __iter__(self):
        text = self.dataset[0].text
        TEXT = self.dataset.fields['text']
        TEXT.eos_token = None
        text = text + ([TEXT.pad_token] * int(math.ceil(len(text) / self.batch_size)
                                              * self.batch_size - len(text)))
        data = TEXT.numericalize(
            [text], device=self.device)
        data = (data
            .stack(("seqlen", "batch"), "flat")
            .split("flat", ("batch", "seqlen"), batch=self.batch_size)
            .transpose("seqlen", "batch")
        )

        dataset = Dataset(examples=self.dataset.examples, fields=[
            ('text', TEXT), ('target', TEXT)])
        while True:
            for i in range(0, len(self) * self.bptt_len, self.bptt_len):
                self.iterations += 1
                seq_len = min(self.bptt_len, len(data) - i - 1)
                yield Batch.fromvars(
                    dataset, self.batch_size,
                    text = data.narrow("seqlen", i, seq_len),
                    target = data.narrow("seqlen", i+1, seq_len),
                )
                         
            if not self.repeat:
                return

In [0]:
# Generate iterators
train_iter, val_iter, test_iter = NamedBpttIterator.splits(
    (train, val, test), batch_size=20, device=torch.device("cuda"), bptt_len=36, shuffle=True, repeat=False)

## Utility functions

In [0]:
def find_ngrams(input_list, n):
  return torch.stack([torch.stack(i) for i in zip(*[input_list[i:] for i in range(n)])])

def make_batch_update(batch, n):
  X = torch.stack([find_ngrams(batch[i,:], n) for i in range(batch.shape[0])])[:,:-1,:]
  y = batch[:,n:]
  return X.contiguous().view(X.shape[0]*X.shape[1], -1), y.contiguous().view(y.shape[0]*y.shape[1])

In [0]:
def calc_prob_interpol(s, unigram_counts, bigram_counts, trigram_counts, alpha):
  prob = alpha[0] * trigram_counts[s[0:2]][s[2]] / bigram_counts[s[0],s[1]] if bigram_counts[s[0],s[1]] else 0 # w0w1w2 / w0w1
  prob += alpha[1] * bigram_counts[s[1],s[2]] / unigram_counts[s[1]] if unigram_counts[s[1]] else 0 # w1w2 / w1
  prob += alpha[2] * unigram_counts[s[2]] / torch.sum(unigram_counts)
  return prob

In [0]:
def calc_ppl_interpol(iterator, unigram_counts, bigram_counts, trigram_counts, alpha):
  total_loss = 0
  total_words = 0

  for b in iter(iterator):
    data = m(torch.transpose(b.text.values, dim0=0, dim1=1))
    X, y = make_batch_update(data, seq_len)
    total_words += X.shape[0]
    for i in range(X.shape[0]):
      s = X[i]
      prob = calc_prob_interpol(s, unigram_counts, bigram_counts, trigram_counts, alpha)
      if prob != 0:
        total_loss += -torch.log(prob)

  return torch.exp(total_loss / total_words)

In [0]:
def make_pred_interpol(sentences, unigram_counts, bigram_counts, trigram_counts, alpha):
  predictions = []

  for i in range(len(sentences)):
    probs = {}

    for _, k in range(len(unigram_counts)):
      s = [TEXT.vocab.stoi[j] for j in sentences[i][8:10]]
      s.extend([k])
      probs[k] = calc_prob_interpol(s, unigram_counts, bigram_counts, trigram_counts, alpha)

    predictions.append([i for i, c in Counter(probs).most_common(20)])

    if i % 100 == 0:
      print(i, '/', len(sentences))
      
  return predictions

In [0]:
def calc_ppl_networks(iterator, net, criterion, model, h0=None): # model = {'nnlm', 'lstm'}
  total_loss = 0
  total_words = 0
  total_correct = 0
  
  for b in iter(iterator):
    if model == 'nnlm':
      data = m(torch.transpose(b.text.values, dim0=0, dim1=1))
      X, y = make_batch_update(data, seq_len)
      prob = net(X)
      loss = criterion(prob, y).detach()
      num_words = X.shape[0]
      pred = torch.argmax(prob, dim=1)
    elif model == 'lstm':
      data = torch.transpose(b.text.values, dim0=0, dim1=1)
      X = data[:,:-1]
      y = data[:,1:]
      prob, _ = net(X, h0)
      loss = criterion(prob.transpose(1,2), y).detach()
      num_words = X.shape[0] * X.shape[1]
      pred = torch.argmax(prob, dim=2)
    else:
      print('error')
    
    total_loss += loss
    total_words += num_words
    total_correct += torch.sum(pred == y).float()
    
  ppl = torch.exp(total_loss / total_words)
  acc = total_correct / total_words

  return ppl, acc

In [0]:
def make_pred_networks(sentences, net, model, h0=None):
  predictions = []

  for i in range(len(sentences)):
    s = torch.tensor([TEXT.vocab.stoi[j] for j in sentences[i][6:10]]).cuda()
    
    if model == 'nnlm':
      prob = net(torch.unsqueeze(s, 0))
    elif model == 'lstm':
      prob, _ = net(torch.unsqueeze(s, 0), h0)
    else:
      print('error')
      
    top_idx = torch.squeeze(torch.argsort(prob, descending=True))[:20]
    predictions.append([TEXT.vocab.itos[j] for j in top_idx])

    if i % 100 == 0:
      print(i, '/', len(sentences))
      
  return predictions

In [0]:
def repackage_hidden(h):
  return tuple(v.detach() for v in h)

## Global variables

In [0]:
seq_len = 3
input_size = 10001
m = nn.ConstantPad1d((seq_len-1, 0), 1)

In [0]:
# load test set
sentences = []
for i, l in enumerate(open("input.txt"), 1):
  sentences.append(re.split(' ', l))

## Model I: Linear interpolation

In [0]:
# get unigram counts (vector)
n_words = TEXT.vocab.__len__()
unigram_count_vector = torch.zeros([n_words, 1], dtype=torch.uint8)

for batch in train_iter:
  unigram_count_vector[batch.text.values, 0] += 1

In [0]:
# get bigram counts (matrix)
bigram_counts_mat = torch.zeros([n_words, n_words], dtype=torch.uint8)

for batch in train_iter:
  for c in range(29): 
    bigram_counts_mat[batch.text.values[c,:], batch.text.values[c+1,:]] += 1

In [0]:
# get trigram counts (dictionary of vectors)
tri_gram = dict()
for i,batch in enumerate(train_iter):
  for c in range(28):
    for i, tup in enumerate(zip(batch.text.values[c,:].cpu().numpy(), batch.text.values[c+1,:].cpu().numpy())):
      if tup in tri_gram:
        # increase the count
        tri_gram[tup][batch.text.values.cpu().numpy()[c+2,i]] += 1
      else:
        # initalize entry
        tri_gram[tup] = torch.zeros([n_words,1], dtype=torch.uint8)
        tri_gram[tup][batch.text.values.cpu().numpy()[c+2,i]] += 1

In [0]:
class linear_model(nn.Module):
  def __init__(self):
    super(linear_model, self).__init__()
    self.alpha = nn.Parameter(torch.Tensor(3, 1))
  def forward(self, unigram_model, bigram_model, trigram_model):
      prob = self.alpha[0] * unigram_model
      prob += self.alpha[1] * bigram_model
      prob += self.alpha[2] * trigram_model
      return prob
  def normalize(self):
      self.alpha.data = torch.clamp(self.alpha.data,0.0)
      self.alpha.data = self.alpha.data / self.alpha.data.sum()

In [0]:
# optimize alphas
model = linear_model()
model.cuda()

optim = torch.optim.Adam(model.parameters())
criterion = torch.nn.NLLLoss()

for i, batch in enumerate(train_iter):
  if i % 100 == 0:
    print(i)
  for c in range(28):
    loss = torch.tensor(0.0,device='cuda')
    for i, tup in enumerate(zip(batch.text.values[c,:].cpu().numpy(),batch.text.values[c+1,:].cpu().numpy())):
      loss = torch.tensor(0.0).cuda()
      bigram_model = bigram_counts_mat[tup[1],:].float().cuda() / bigram_counts_mat[tup[1],:].float().sum().cuda()
      trigram_model = tri_gram[tup].squeeze().float().cuda() / tri_gram[tup].squeeze().float().sum().cuda()
      prob = model(unigram_model.squeeze(), bigram_model.squeeze(), trigram_model.squeeze())
      loss += criterion(prob.reshape(1,-1), batch.text.values[c+2,i].reshape(-1))
    loss.backward()
    optim.step()
    model.normalize()
    optim.zero_grad()
    
print(model.alpha.data)

In [0]:
# calc train ppl and val ppl
train_ppl = calc_ppl_interpol(train_iter, unigram_count_vector, bigram_counts_matrix, tri_gram, model.alpha.data)
print('Train PPL:', train_ppl)

val_ppl = calc_ppl_interpol(val_iter, unigram_count_vector, bigram_counts_matrix, tri_gram, model.alpha.data)
print('Val PPL:', val_ppl)

In [0]:
# generate predictions for kaggle
predictions = make_pred_interpol(sentences, unigram_count_vector, bigram_counts_matrix, tri_gram, model.alpha.data)

## Model II: NNLM

In [0]:
# language modeling with distributed representations
class Bennet(torch.nn.Module):
  def __init__(self, input_size, embed_size, hidden_size, seq_len):
    super(Bennet, self).__init__()
    self.embed = nn.Embedding(input_size, embed_size)
    self.hidden = nn.Linear(seq_len * embed_size, hidden_size, bias=True)
    self.fc = nn.Linear(hidden_size, input_size, bias=True)
    
  def forward(self, inputs):
    x = self.embed(inputs) 
    x = x.view(x.shape[0], x.shape[1] * x.shape[2])
    
    y = self.hidden(x) 
    y = torch.tanh(y) 

    z = self.hidden(x)
    output = self.fc(z) + self.fc(y) 

    return output

In [0]:
# initalize NN, optimizer, loss function
embed_size = 60
hidden_size = 50
seq_len = 4

learning_rate = 1e-2
num_epochs = 20

net = Bennet(input_size, embed_size, hidden_size, seq_len)
net.cuda()

optimizer = torch.optim.Adam(net.parameters(), lr=learning_rate, weight_decay=1e-3)
lr_lambda = lambda t: learning_rate / (1 + t * 1e-7)
scheduler = torch.optim.lr_scheduler.LambdaLR(optimizer, lr_lambda, last_epoch=-1)
criterion = torch.nn.CrossEntropyLoss(reduction = 'sum')

In [0]:
# train model
for i in range(num_epochs):
  scheduler.step()
  
  for b in iter(train_iter):
    optimizer.zero_grad()
    data = m(torch.transpose(b.text.values, dim0=0, dim1=1))
    X, y = make_batch_update(data, seq_len)
    prob = net(X)
    loss = criterion(prob, y)
    loss.backward()
    optimizer.step()
  
  ppl, acc = calc_ppl_networks(train_iter, net, criterion, 'nnlm')
  ppl_val, acc_val = calc_ppl_networks(val_iter, net, criterion, 'nnlm')
  print('Epoch: %d, Train PPL: %.4f, Train Acc: %.4f, Val PPL: %.4f, Val Acc: %.4f' % (i, ppl, acc, ppl_val, acc_val))

In [0]:
# generate predictions for kaggle
predictions = make_pred_networks(sentences, net, 'nnlm')

## Model II: LSTM

In [0]:
# LSTM RNN
class RNNet(torch.nn.Module):

  def __init__(self, input_size, hidden_size, num_layers, dropout=0.5, weight_tie=False, weight_init=0.05):
    super(RNNet, self).__init__()
    self.emb = torch.nn.Sequential(torch.nn.Embedding(input_size, hidden_size), torch.nn.Dropout(dropout))
    self.rnn = torch.nn.LSTM(input_size=hidden_size, hidden_size=hidden_size, num_layers=num_layers, bias=True, batch_first=True, dropout=dropout)
    self.lnr = torch.nn.Sequential(torch.nn.Dropout(dropout), torch.nn.Linear(hidden_size, input_size))
    
    for f in self.parameters():
      torch.nn.init.uniform_(f, a=-weight_init, b=weight_init)
      
    if weight_tie == True:
      self.lnr[1].weight.data=self.emb[0].weight.data
      
  def forward(self, inputs, h0=None):
    x = self.emb(inputs) # batch x seqlen x hidden
    x, hidden = self.rnn(x, h0) # batch x seqlen x hidden
    y = self.lnr(x) # batch x seqlen x vocab
    return y, hidden

In [0]:
# initalize LSTM RNN, optimizer, loss function
hidden_size = 650
num_layers = 2
dropout = 0.5
learning_rate = 1
num_epochs = 39

net = RNNet(input_size, hidden_size, num_layers, dropout,weight_tie=True)
net.cuda()

optimizer = torch.optim.SGD(net.parameters(), lr=learning_rate)
lr_lambda = lambda t: learning_rate / (1.2**max(t-6,0))
scheduler = torch.optim.lr_scheduler.LambdaLR(optimizer, lr_lambda, last_epoch=-1)
criterion = torch.nn.CrossEntropyLoss(reduction = 'sum')

In [0]:
# run LSTM RNN
for i in range(num_epochs):
  scheduler.step()
  h0 = None

  for b in iter(train_iter):
    optimizer.zero_grad()
    data = torch.transpose(b.text.values, dim0=0, dim1=1)
    X = data[:,:-1]
    y = data[:,1:]
    if X.shape[1] == 35:
      prob, h = net(X, h0)
      h0 = repackage_hidden(h)
      loss = criterion(prob.transpose(1,2), y)
      loss.backward()
      clip_grad_norm_(net.parameters(), max_norm=5)
      optimizer.step()
      
  ppl, acc = calc_ppl_networks(train_iter, net, criterion, 'lstm', h0)
  ppl_val, acc_val = calc_ppl_networks(val_iter, net, criterion, 'lstm', h0)
  print('Epoch: %d, Train PPL: %.4f, Train Acc: %.4f, Val PPL: %.4f, Val Acc: %.4f' % (i, ppl, acc, ppl_val, acc_val))

In [0]:
# generate predictions for kaggle
predictions = make_pred_networks(sentences, net, 'lstm', h0)

## Export predictions

In [0]:
# mount drive
from google.colab import drive
drive.mount('/content/gdrive')

In [0]:
with open("/content/gdrive/My Drive/predictions2.txt", "w") as f:
  f.write("id,word\n")
  for i, l in enumerate(open("input.txt"), 1):
    f.write("%d,%s\n"%(i, " ".join(predictions[i-1])))